In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
# Abiertos

parser_pag = parser('http://www.fapemat.mt.gov.br/vigentes')

#general

links_abiertos = parser_pag.xpath('//div[@class="news-item"]//h3/a/@href')

titulos_abiertos = parser_pag.xpath('//div[@class="news-item"]//h3/a/text()')

descripcion_abiertos = []
fecha_abiertos = []
pdfs_abiertos = []

#proyecto por proyecto

for link in links_abiertos:
    parser_proy = parser(link)

    # Descripción 
    descripcion_proy_bruto = parser_proy.xpath('//div[@class="edital_conteudo"]//p//text()')

    descripcion_proy = ''
    for desc in descripcion_proy_bruto:
        desc = desc.strip()
        if desc != '':
            descripcion_proy = descripcion_proy + desc + ' '

    # Fecha
    fecha = parser_proy.xpath('//div[@class="edital_data"]//text()')[0]

    #Pdfs
    pdfs_proy_brutos = parser_proy.xpath('//div[@class="journal-content-article"]//a/@href')

    link_base = 'http://www.fapemat.mt.gov.br'
    
    pdfs_proy = ''
    for pdf in pdfs_proy_brutos:
        pdfs_proy = pdfs_proy + link_base + pdf + ', '

    descripcion_abiertos.append(descripcion_proy)
    fecha_abiertos.append(fecha)
    pdfs_abiertos.append(pdfs_proy)

fapemat_abiertos = pd.DataFrame()
fapemat_abiertos['Titulo'] = titulos_abiertos
fapemat_abiertos['Link'] = links_abiertos
fapemat_abiertos['Descripción'] = descripcion_abiertos
fapemat_abiertos['Fecha'] = fecha_abiertos
fapemat_abiertos['Pdfs'] = pdfs_abiertos
fapemat_abiertos['Estado'] = 'Abierto'


In [4]:
# CErrados

parser_pag = parser('http://www.fapemat.mt.gov.br/encerrados')

links_cerrados = parser_pag.xpath('//div[@class="news-item"]//h3/a/@href')

titulos_cerrados = parser_pag.xpath('//div[@class="news-item"]//h3/a/text()')

descripcion_cerrados = []
fecha_cerrados = []
pdfs_cerrados = []

#proyecto por proyecto

for link in links_cerrados:
    parser_proy = parser(link)

    # Descripción 
    descripcion_proy_bruto = parser_proy.xpath('//div[@class="edital_conteudo"]//p//text()')

    descripcion_proy = ''
    for desc in descripcion_proy_bruto:
        desc = desc.strip()
        if desc != '':
            descripcion_proy = descripcion_proy + desc + ' '

    # Fecha
    fecha = parser_proy.xpath('//div[@class="edital_data"]//text()')[0]

    #Pdfs
    pdfs_proy_brutos = parser_proy.xpath('//div[@class="journal-content-article"]//a/@href')

    link_base = 'http://www.fapemat.mt.gov.br'
    
    pdfs_proy = ''
    for pdf in pdfs_proy_brutos:
        pdfs_proy = pdfs_proy + link_base + pdf + ', '

    descripcion_cerrados.append(descripcion_proy)
    fecha_cerrados.append(fecha)
    pdfs_cerrados.append(pdfs_proy)


fapemat_cerrados = pd.DataFrame()
fapemat_cerrados['Titulo'] = titulos_cerrados
fapemat_cerrados['Link'] = links_cerrados
fapemat_cerrados['Descripción'] = descripcion_cerrados
fapemat_cerrados['Fecha'] = fecha_cerrados
fapemat_cerrados['Pdfs'] = pdfs_cerrados
fapemat_cerrados['Estado'] = 'Cerrado'

In [5]:


fapemat = pd.concat([fapemat_abiertos, fapemat_cerrados])

In [7]:
fapemat.to_excel('brasil_fapemat.xlsx')